In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from time import time
import glob
# sns.set()

In [28]:
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET
def pretty_xml(element, indent, newline, level=0):  # elemnt为传进来的Elment类，参数indent用于缩进，newline用于换行
    if element:  # 判断element是否有子元素    
        if (element.text is None) or element.text.isspace():  # 如果element的text没有内容
            element.text = newline + indent * (level + 1)
        else:
            element.text = newline + indent * (level + 1) + element.text.strip() + newline + indent * (level + 1)
            # else:  # 此处两行如果把注释去掉，Element的text也会另起一行
            # element.text = newline + indent * (level + 1) + element.text.strip() + newline + indent * level
    temp = list(element)  # 将element转成list
    for subelement in temp:
        if temp.index(subelement) < (len(temp) - 1):  # 如果不是list的最后一个元素，说明下一个行是同级别元素的起始，缩进应一致
            subelement.tail = newline + indent * (level + 1)
        else:  # 如果是list的最后一个元素， 说明下一行是母元素的结束，缩进应该少一个    
            subelement.tail = newline + indent * level
        pretty_xml(subelement, indent, newline, level=level + 1)  # 对子元素进行递归操作

In [19]:
files=glob.glob(r"C:\Users\poac\Desktop\towjx\FMPayload\TGTH\TGTHYY*\XML\FMxml.xml")
for file in files:
    print('读入xml数据:', file, )
    tree = ET.ElementTree(file=file)     #载入数据
    root = tree.getroot()     #获取根节点
    filename=file.split('\\')[-3]
    #获取所需字段数据作为原始文件
    final0=[]
    for i, rooti in enumerate(root):
        if i==79:
            continue
        FaultKnowledgeName=rooti[2].text
#         mid=[1, 0, 0, 1]
        child=rooti[5][0][0]
        Criterias=child.get('Criterias')
        for j, rootj in enumerate(child):
            num=rootj[0].text
            name=rootj[1].text
            cond=rootj[3].text      #状态
            normal1=rootj[4].text    #参数正常值
            fault=rootj[5].text    #参数故障值
            # 1去空格; 2 ≤统一表示为<=; 3≥统一表示为>=;
            normal1=normal1.replace(' ', '')
            normal1=normal1.replace('=<', '<=')
            normal1=normal1.replace('=>', '>=')
    #         fault=fault.replace(' ', '')
            fault=fault.split('s ')[-1]
            fault=fault.replace('x or', 'x')
            fault=fault.replace('=<', '<=')
            fault=fault.replace('=>', '>=')  
            mid=[FaultKnowledgeName, Criterias, num, name, cond, normal1, fault]
            final0.append(mid)     
    final0=pd.DataFrame(final0)
    final0.columns=['故障模式名称', '故障判据', '参数代号', '参数名称', '参数状态标识', 'IDS表正常值', '载荷提供故障值']

    final=final0.copy()
    normal_IDS=final['IDS表正常值'].values.tolist()
    lim_IDS=final['IDS表正常值'].values.tolist()
    fault_loads=final['载荷提供故障值']
    lim_loads=final['载荷提供故障值'].values.tolist()
    
    normal=final['载荷提供故障值'].values.tolist()    
    anomaly_section1=final['载荷提供故障值'].values.tolist()
    anomaly_section2=final['载荷提供故障值'].values.tolist()
    
    for i, faulti in enumerate(fault_loads):
        if ('x' in faulti) and ('H' not in faulti) and ('m' not in faulti) and ('b' not in faulti):
            lim_IDS[i]=Expression2Limit(normal_IDS[i])   
            lim_loads[i]=Complement(Expression2Limit(faulti))    
            if IsSubset(Expression2Limit(faulti), lim_IDS[i])==1:
                normal[i]=lim_IDS[i]
            else:
                normal[i]=Intersection(lim_IDS[i], lim_loads[i])
            anomaly_section1[i]=Intersection(Complement(lim_IDS[i]), lim_loads[i])
            if IsSubset(Expression2Limit(faulti), lim_IDS[i])==1:
                anomaly_section2[i]=anomaly_section1[i]
            else:
                anomaly_section2[i]=Expression2Limit(faulti)
#             normal[i]=Const2Expre(normal[i])
#             anomaly_section1[i]=Const2Expre(anomaly_section1[i])
#             anomaly_section2[i]=Const2Expre(anomaly_section2[i])
#             if i==52:
#         break
#     final['参数正常值2']=normal2
    final['正常区间']=normal
    final['1级异常区间']=anomaly_section1
    final['2级异常区间']=anomaly_section2
    
    final.sort_values(['参数代号'], inplace=True)
    final.to_csv(filename+'_0.csv', index=None)
    final=trim_csv(final)
    final.reset_index(drop=True, inplace=True)
    final.to_csv(filename+'_1.csv', index=None)

    # final.sort_values(['参数代号', '参数状态标识'], ascending=[1, 1], inplace=True)
    # final.reset_index(drop=True, inplace=True)
    # final.to_csv('final.csv', index=None)
    # final=final[['参数代号', '参数状态标识', '参数正常值']]

    diff_num=np.unique(final['参数代号'])
    result=[]
    for num_i in diff_num: 
        mid1=final[final['参数代号']==num_i]
        diff_biaoshi=np.unique(mid1['参数状态标识'])
        for biaoshi_i in diff_biaoshi:
            mid2=mid1[mid1['参数状态标识']==biaoshi_i]
            mid=mid2.head(1)
            mid['正常区间']=[Intersections(mid2['正常区间'].values[0])]
            mid['2级异常区间']=[UnionSets(mid2['2级异常区间'].values[0])]
            mid['1级异常区间']=[Complement(UnionSet(mid['正常区间'].values[0],  mid['2级异常区间'].values[0]))]
            result.append(mid)
    result=pd.concat(result)
    # result=result[result['参数正常值']!=[]]
    result.reset_index(drop=True, inplace=True)
#     a.to_csv(filename+'.csv', index=None)
    result=result[['故障模式名称', '故障判据', '参数代号', '参数名称', '参数状态标识', '正常区间', '1级异常区间', '2级异常区间']]
    result.to_csv(filename+'_2.csv', index=None)
    
    result['正常区间']=[Limit2Expression(lim) for lim in result['正常区间'].values]
    result['1级异常区间']=[Limit2Expression(lim) for lim in result['1级异常区间'].values]
    result['2级异常区间']=[Limit2Expression(lim) for lim in result['2级异常区间'].values]
    
    result.to_csv(filename+'_3.csv', index=None)

读入xml数据: C:\Users\poac\Desktop\towjx\FMPayload\TGTH\TGTHYY0101\XML\FMxml.xml
读入xml数据: C:\Users\poac\Desktop\towjx\FMPayload\TGTH\TGTHYY0102\XML\FMxml.xml
读入xml数据: C:\Users\poac\Desktop\towjx\FMPayload\TGTH\TGTHYY0201\XML\FMxml.xml
读入xml数据: C:\Users\poac\Desktop\towjx\FMPayload\TGTH\TGTHYY0301\XML\FMxml.xml


d:\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\Anaconda3\lib\site-packages\ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

读入xml数据: C:\Users\poac\Desktop\towjx\FMPayload\TGTH\TGTHYY0401\XML\FMxml.xml
读入xml数据: C:\Users\poac\Desktop\towjx\FMPayload\TGTH\TGTHYY0501\XML\FMxml.xml
读入xml数据: C:\Users\poac\Desktop\towjx\FMPayload\TGTH\TGTHYY1701\XML\FMxml.xml


In [20]:
xml_files=glob.glob(r"C:\Users\poac\Desktop\towjx\IDS27_RADW\TGTHYY*.xml")
csv_files=glob.glob(r".\*3.csv")
parameters=[]
for xml_file in xml_files:
    print('读入xml数据:', xml_file, ) 
    ET.register_namespace("", "IDSInsertEvents")
    tree = ET.ElementTree(file = xml_file)     #载入数据
    root = tree.getroot()     #获取根节点.shape
    # ET.dump(tree)
    
    for i, Action_i in enumerate(root[2]):
        if len(Action_i)<=3:   #2021-8-1 wjx              
            print('len(Action_i)<=3')
            continue
        for j, Criterion_j in enumerate(Action_i[3]):
            for k, parameter_k in enumerate(Criterion_j[1]):
                index_ijk=[parameter_k[0].text, parameter_k[1].text,  parameter_k[3].text, parameter_k[-1].text]
                count=0
                for csv_file in csv_files:
                    count+=1
                    the_csv=pd.read_csv(csv_file, index_col=0, encoding='utf-8')       
                    the_csv['参数代号']=[i[:-2] for i in the_csv['参数代号'].values]    
#         the_csv=pd.read_csv(csv_file, index_col=0, encoding='gbk')
#                     index_ijk=[parameter_k[0].text, parameter_k[1].text,  parameter_k[-2].text, parameter_k[-1].text]   #wrong!
                    mid=the_csv[(the_csv['参数代号']==index_ijk[0]) & (the_csv['参数状态标识']==index_ijk[2])]
        #             print([i, j, k], index_ijk)
                    if count==1:
                        ET.SubElement(parameter_k, 'NormalValues').text=parameter_k[-1].text
                        ET.SubElement(parameter_k, 'AnomalyValues').text='/'
                        ET.SubElement(parameter_k, 'FaultValues').text='/'             
                        parameter_k.remove(parameter_k[4]) 
#                         print([-1, parameter_k[0].text, parameter_k[1].text, parameter_k[3].text])
                    if mid.shape[0]>0:
                        print([csv_file, parameter_k[0].text, parameter_k[1].text, parameter_k[3].text])
                        parameter_k[-3].text=mid['正常区间'].values[0]
                        parameter_k[-2].text=mid['1级异常区间'].values[0]
                        parameter_k[-1].text=mid['2级异常区间'].values[0]
                        new_index_ijk=[parameter_k[0].text, parameter_k[1].text, parameter_k[-3].text, parameter_k[-2].text, parameter_k[-1].text]   
                parameter=[i.text for i in parameter_k];                  
                parameters.append(parameter);
    
    newname='new_'+ xml_file.split('\\')[-1]
    tree=ET.ElementTree(root)  # root为修改后的root
    pretty_xml(root, '\t', '\n')  # 执行美化方法
    tree.write("C:\\Users\\poac\\Desktop\\result_xml_2021-8-2\\"+newname, encoding='utf-8')
parameters=pd.DataFrame(parameters);
parameters.columns=['ParameterNo', 'ParameterName', 'ValueType', 'Description', 'NormalValues','AnomalyValues','FaultValues'];
parameters.sort_values(['ParameterNo'], inplace=True)
# parameters.drop_duplicates(inplace=True)
parameters.to_csv('parameters.csv', index=None)

读入xml数据: C:\Users\poac\Desktop\towjx\IDS27_RADW\TGTHYY0101_1_27_001_RADW.xml
读入xml数据: C:\Users\poac\Desktop\towjx\IDS27_RADW\TGTHYY0102_1_27_001_RADW.xml
len(Action_i)<=3
读入xml数据: C:\Users\poac\Desktop\towjx\IDS27_RADW\TGTHYY0201_27_RADW.xml
['.\\TGTHYY0201_3.csv', 'DCEH1418/1', '人系统研究机柜100V电压遥测', '加电']
['.\\TGTHYY0201_3.csv', 'DCEH1418/1', '人系统研究机柜100V电压遥测', '加电']
['.\\TGTHYY0201_3.csv', 'DCEH1418/1', '人系统研究机柜100V电压遥测', '加电']
['.\\TGTHYY0201_3.csv', 'DCEH1418/1', '人系统研究机柜100V电压遥测', '加电']
['.\\TGTHYY0201_3.csv', 'DCEH1418/1', '人系统研究机柜100V电压遥测', '加电']
['.\\TGTHYY0201_3.csv', 'DCEH1418/1', '人系统研究机柜100V电压遥测', '加电']
['.\\TGTHYY0201_3.csv', 'DCEH1418/1', '人系统研究机柜100V电压遥测', '加电']
['.\\TGTHYY0201_3.csv', 'DCEH1418/1', '人系统研究机柜100V电压遥测', '加电']
读入xml数据: C:\Users\poac\Desktop\towjx\IDS27_RADW\TGTHYY0301_27001_RADW.xml
['.\\TGTHYY0301_3.csv', 'DCEH2567/1', '大气1机电子流检测A', '大气1机传感器A入轨加电']
['.\\TGTHYY0301_3.csv', 'DCEH2567/1', '大气1机电子流检测A', '大气1机传感器A入轨断电']
['.\\TGTHYY0301_3.csv', 'DCEH2570/1', '大气1机电

# 脚本函数

In [25]:
def Expression2Limit(string):
    '''
    description: 
    求约束区间
    string：'x>=10', 'x>10 or x<100....', ...
    '''
    lim=[]
    count_or=string.count('or')
    if count_or>=1:
        for i in string.split(' or '):
            lim.append(_Expression2Limit_single(i))
    else:
        lim=_Expression2Limit_single(string)
    return lim
def _Expression2Limit_single(string):
    '''
    description: 
    *此函数不建议单独使用
    求约束区间，区间必须一段区间，不允许包含or
    string：'x>=10', 'x<100....', ...
    '''
    string=string.replace(' ', '')
    string=string.replace('=<', '<=')
    string=string.replace('=>', '>=')
    # 仅仅针对单个区间，不要包含or多个区间，否则触发异常
    # 同时包含上下限时不允许写成10>x>0,必须写成0<x<10，否则出发异常
    lim=[-float('inf'), float('inf'), 0, 0]
    count1=string.count('>')
    count2=string.count('<')
    count3=string.count('=')
    count4=string.count('or')
    if (count4>=1) or (count1==2) or (count1>=1 and count2>=1) or (count3>2):
        raise Exception('Single interval should be provided!')
    if count1==1:
        if count3==1:  #x>=10
            lim[0]=float(string.split('>=')[-1])
            lim[2]=1
        else:    #x>10
            lim[0]=float(string.split('>')[-1])
            lim[2]=0
    elif count2==1:
        if count3==1:  #x<=10
            lim[1]=float(string.split('<=')[-1])
            lim[3]=1
        else:
            lim[1]=float(string.split('<')[-1])
            lim[3]=0
    else:   #count2=2
        lim[0]=float(string.split('<')[0])
        lim[1]=float(string.split('<')[-1].split('=')[-1])
        if '=' in string.split('<')[1]:  # x<=10(...)20
            lim[2]=1
        if '=' in string.split('<')[2]:  # x(...)10<=20
            lim[3]=1
    return lim

In [127]:
# 约束区间翻译为区间描述
def Limit2Expression(constraints):
    '''
    description: 
    constraints:若单个区间则constraints为1维列表; 若多个区间并且为or的关系，则constraints为2维列表
    约束区间[0, 10, 1, 0]翻译为0<=x<10
    约束区间[[0, 10, 1, 0], [20, 30, 1, 1],...]翻译为0<=x<10 or 20<=x<=30...
    '''
    mid=[]
    dim=np.array(constraints).ndim
    if dim==1:
        expres=_Limit2Expression_single(constraints)
    else:
        for constraint in constraints:
            mid.append(_Limit2Expression_single(constraint))
        if len(mid)==1:
            expres=mid[0]
        else:
            expres=mid[0]
            for mid_i in mid[1:]:
                expres=expres+' or '+mid_i
    return expres    
def _Limit2Expression_single(constraints):
    '''
    description: 
    *此函数不建议单独使用
    约束区间[0, 10, 1, 0]翻译为0<=x<10 
    constraints：必须写成1维列表形式[0, 10, 1, 0]否则触发异常
    '''
    if np.array(constraints).ndim>1:
        raise Exception('The input parameter should be a 1D list!')
    elif constraints==[]:
        expre=[]
    else: 
        if constraints[0]==-float('inf'):
            if constraints[3]==0:
                expre='x'+'<'+str(constraints[1])
            else:
                expre='x'+'<='+str(constraints[1])        
        elif constraints[1]==float('inf'):  
            if constraints[2]==0:
                expre='x'+'>'+str(constraints[0])
            else:
                expre='x'+'>='+str(constraints[0])  
        else:
            if constraints[2]==0:
                expre1=str(constraints[0])+'<'
            else:
                expre1=str(constraints[0])+'<='
            if constraints[3]==0:
                expre2='<'+str(constraints[1])
            else:
                expre2='<='+str(constraints[1])
            expre=expre1+'x'+expre2
    return expre

In [ ]:
def Limit_list2string(lim):
    '''
    description: 
    约束区间转换为表达式
    lim=[1,2,0,1]/[[1,2,1,1], [4,5,0,1],...]
    result='(1, 2]'/'[1, 2] or (4, 5]'
    '''
    result=''
    if lim==[]:
        result=[]
    elif len(lim)==4:    # np.array(lists).ndim=1
        if lim[2]==0:
            mid0='('
        else:
             mid0='['
        if lim[3]==0:
            mid3=')'
        else:
             mid3=']'
        result=mid0+str(lim[0])+', '+str(lim[1])+mid3
    else:
        for i, lim_i in enumerate(lim):
            if lim_i[2]==0:
                mid0='('
            else:
                 mid0='['
            if lim_i[3]==0:
                mid3=')'
            else:
                 mid3=']'
            if i==0:
                result+=(mid0+str(lim_i[0])+', '+str(lim_i[1])+mid3)
            else:
                result+=' or ' + (mid0+str(lim_i[0])+', '+str(lim_i[1])+mid3)         
    return result

In [8]:
def trim_csv(x):
    mid=x['载荷提供故障值'].values
    index=[]
    for i in mid:
        index.append('x' in i)
    return x[index] 

In [10]:
def IsSubset(lim1, lim2):
    '''
    description: 
    判断lim1是否为lim2的子集
    
    单个区间直接给出lim0=[0, 10, 1, 1],或者[[0, 10, 1, 1]](不推荐)
    多个区间必须以2D列表给出, [[0, 10, 1, 1], [20, 30, 1, 1]]
    
    lim1：单区间[1, 10, 1, 1]或区间列表[[1, 10, 1, 1], [18, 20, 1, 1]]
    lim2：单区间[0, 12, 1, 1]或区间列表[[0, 12, 1, 1], [15, 20, 1, 1]]
    当lim1所有区间都是lim2的子集时,输出1
    '''
    return Intersections([lim1, lim2])==lim1   

def IsSubset(lim1, lim2):
    '''
    description: 
    判断lim1是否为lim2的子集
    
    单个区间直接给出lim0=[0, 10, 1, 1],或者[[0, 10, 1, 1]](不推荐)
    多个区间必须以2D列表给出, [[0, 10, 1, 1], [20, 30, 1, 1]]
    
    lim1：单区间[1, 10, 1, 1]或区间列表[[1, 10, 1, 1], [18, 20, 1, 1]]
    lim2：单区间[0, 12, 1, 1]或区间列表[[0, 12, 1, 1], [15, 20, 1, 1]]
    当lim1所有区间都是lim2的子集时,输出1
    '''
    allarein=0
    if np.array(lim1).ndim==1:
        lim1=[lim1]
    if np.array(lim2).ndim==1:
        lim2=[lim2]
    for lim1_i in lim1:
        for lim2_i in lim2:
            if (lim1_i[0]<lim2_i[0]) or (lim1_i[1]>lim2_i[1]) or (lim1_i[0]==lim2_i[0] and lim1_i[2]>lim2_i[2]) or (lim1_i[1]==lim2_i[1] and lim1_i[3]>lim2_i[3]):
                isin=0
            else:
                isin=1
            allarein+=isin   
    if allarein==len(lim1):
        allarein=1
    else:
        allarein=0
    return allarein          

In [222]:
def Complement(lim0):
    '''
    description: 
    求区间补集
    单个区间直接给出lim0=[0, 10, 1, 1],或者[[0, 10, 1, 1]](不推荐)
    多个区间必须以2D列表给出, [[0, 10, 1, 1], [20, 30, 1, 1]]
    '''
    dim=np.array(lim0).ndim
    lim=[]
    if (dim==2) and (len(lim0)==1):
        lim0=lim0[0]
        dim=np.array(lim0).ndim
    if lim0==[]:
        lim=[-float('inf'), float('inf'), 0, 0]
    elif dim==1:
        if lim0[0]==-float('inf'):
            if lim0[1]==float('inf'):
                True
            else:
                lim=[lim0[1], float('inf'), 1-lim0[3], 0]
        elif lim0[1]==float('inf'):
                lim=[-float('inf'), lim0[0], 0, 1-lim0[2]]
        else:
                lim.append([-float('inf'), lim0[0], 0, 1-lim0[2]])
                lim.append([lim0[1], float('inf'), 1-lim0[3], 0])         
    else:
        for i, lim_i in enumerate(lim0[:-1]):
            if i==0:
                if lim_i[0]==-float('inf'):
                    True
                else:
                    lim.append([-float('inf'), lim_i[0], 0, 1-lim_i[2]])
            lim.append([lim_i[1], lim0[i+1][0], 1-lim_i[3], 1-lim0[i+1][2]])
            
        lim_i=lim0[-1]
        if lim_i[1]==float('inf'):
            True
        else:
            lim.append([lim_i[1], float('inf'), 1-lim_i[3], 0])
    if len(lim)==1:
        lim=lim[0]
    return lim

In [12]:
#列表表达式取交集                            
def Intersections(limits, another_limits=None):
    '''
    description: 
    多个区间求交集, limits为嵌套列表形式，每个元素为约束区间, 每个约束区间不限制单区间(1/2维列表皆可)
    
    单个区间直接给出lim0=[0, 10, 1, 1],或者[[0, 10, 1, 1]](不推荐)
    多个区间必须以2D列表给出, [[0, 10, 1, 1], [20, 30, 1, 1]]
    
    列表每个元素内部(每个区间之间)的关系为or, 列表元素之间的关系为and
    limits：[[[0, 10, 1, 1], [15, 30, 1, 1]], [5, 20, 0, 0]]
    tips:
    如果同时给出another_limits(不推荐),则会求limits和another_limits的交集
    '''
    if another_limits!=None:
        limits=[limits, another_limits]
    if np.array(limits).ndim==1:
        limits=[limits]
    limit=[-float('inf'), float('inf'), 1, 1]
    for limit_i in limits:
        if type(limit_i)==list:
            limit=Intersection(limit, limit_i)
#         else:   #如果limits中包含非list值处理方式  忽略?
#             limit=[]
    if limit==[-float('inf'), float('inf'), 1, 1]:
        limit=[]
    return limit
def Intersection(limit1, limit2):
    '''
    description: 
    两个区间求交集, 不限制单区间(1/2维列表皆可)
        
    单个区间直接给出lim0=[0, 10, 1, 1],或者[[0, 10, 1, 1]](不推荐)
    多个区间必须以2D列表给出, [[0, 10, 1, 1], [20, 30, 1, 1]]
    
    limit1：[0, 10, 1, 1]/[[0, 10, 1, 1], [15, 30, 1, 1]]
    limit2：[5, 20, 0, 0]/...
    '''
    lim=[]
    dim1, dim2 = np.array(limit1).ndim, np.array(limit2).ndim
    if ([] in limit1) or ([] in limit2):
        True
    elif dim1+dim2==2:
        lim=_Intersection_singles(limit1, limit2)
    elif dim1+dim2==3:
        if dim1==1:
            for limit2_i in limit2:
                mid=_Intersection_singles(limit1, limit2_i)
                if mid!=[]: 
                    lim.append(mid)
        else:
            for limit1_i in limit1:
                mid=_Intersection_singles(limit1_i, limit2)
                if mid!=[]: 
                    lim.append(mid)
    else: #dim1=2 dim2=2
        for limit1_i in limit1:
            for limit2_i in limit2:
                mid=_Intersection_singles(limit1_i, limit2_i)
                if mid!=[]: 
                    lim.append(mid)
    if len(lim)==1:
        lim=lim[0]
    return lim
def _Intersection_singles(limit1, limit2):
    '''
    description: 
    *此函数不建议单独使用
    两个单区间求交集, 必须为单区间(1维列表)否则触发异常
       
    单个区间直接给出lim0=[0, 10, 1, 1],或者[[0, 10, 1, 1]](不推荐)
    多个区间必须以2D列表给出, [[0, 10, 1, 1], [20, 30, 1, 1]]
    
    limit1：[0, 10, 1, 1]
    limit2：[5, 20, 0, 0]
    '''
    lim=[]
    lim1, lim2=limit1, limit2
    dim1, dim2 = np.array(lim1).ndim, np.array(lim2).ndim  
    if (dim1==2) and (len(limit1)==1):
        lim1=limit1[0]
        dim1 = np.array(lim1).ndim
    if (dim2==2) and (len(limit2)==1):
        lim2=limit2[0]
        dim2 = np.array(lim2).ndim
    
    if dim1+dim2==2:
        if lim1==[] or lim2==[]:
            lim=[]
        elif (lim1[0]>lim2[1]) or (lim2[0]>lim1[1]):
                True
        else:
            lim=[-float('inf'), float('inf'), 0, 0]
            mins=max([lim1[0], lim2[0]])
            maxs=min([lim1[1], lim2[1]])
            lim[0]=mins
            lim[1]=maxs
            if mins==lim2[0]:
                lim[2]=lim2[2]
            else:
                lim[2]=lim1[2]
            if maxs==lim2[1]:
                lim[3]=lim2[3]
            else:
                lim[3]=lim1[3]   
            
            if lim[0]==lim[1]:
                if lim[2]+lim[3]==2:   #lim[2]=1 and lim[3]=1
#                     lim=[lim[0]]
                    True
                else:
                    lim=[]
    else:
        raise Exception('Single intervals should be provided!')
    return lim

In [221]:
# def _GetUnionSet_singles(limit1, limit2):
#     '''
#     description: 
#     *此函数不建议单独使用
#     两个单区间求交集, 必须为单区间(1维列表)否则触发异常
#     limit1：[0, 10, 1, 1]
#     limit2：[5, 20, 0, 0]
#     '''          
#     lim=[]
#     lim1, lim2=limit1, limit2
#     dim1, dim2 = np.array(limit1).ndim, np.array(limit2).ndim
#     if dim1+dim1==2:
#         if lim1==[]:
#             lim=lim2
#         elif lim2==[]:
#             lim=lim1
#         else:
#             if GetIntersections(lim1, lim2)==[]:
#                 lim=[lim1, lim2]
#             else:   
#                 lim=[-float('inf'), float('inf'), 0, 0]
#                 mins=min([lim1[0], lim2[0]])
#                 maxs=max([lim1[1], lim2[1]])
#                 lim[0]=mins
#                 lim[1]=maxs
#                 if mins==lim1[0]:
#                     lim[2]=lim1[2]
#                 else:
#                     lim[2]=lim2[2]
#                 if maxs==lim1[1]:
#                     lim[3]=lim1[3]
#                 else:
#                     lim[3]=lim2[3]
#                 if lim[0]==lim[1]:
#                     if lim[2]==lim[3]:
#     #                     lim=[lim[0]]
#                         True
#                     else:
#                         lim=[]
#     else:
#         raise Exception('Single intervals should be provided!')
#     return lim

# def _GetUnionSet_singles_multiple(limit1, limit2)
#     '''
#     description: 
#     *此函数不建议单独使用
#     单区间和多区间求交集
#     limit1：[0, 10, 1, 1]
#     limit2：[5, 20, 0, 0]/[[0, 1, 1, 1], [5, 20, 0, 0]]
#     '''  
#     if ([] in limit1) or ([] in limit2):
#         raise Exception('[] should not in the given limitations!')
#     lim=[]
#     lim1, lim2=limit1, limit2
#     dim1, dim2 = np.array(limit1).ndim, np.array(limit2).ndim
#     if dim1==2:
#         raise Exception('parameter 1 should be 1D list!')
#     if dim1+dim2==2:
#         lim=_GetUnionSet_singles(limit1, limit2)
#     else:   # dim1=1, dim2=2

def UnionSets(limits, another_limits=None):
    '''
    description: 
    多个区间求并集, limits为嵌套列表形式，每个元素为约束区间, 每个约束区间不限制单区间(1/2维列表皆可)
    
    单个区间直接给出lim0=[0, 10, 1, 1],或者[[0, 10, 1, 1]](不推荐)
    多个区间必须以2D列表给出, [[0, 10, 1, 1], [20, 30, 1, 1]]
    
    列表每个元素内部(每个区间之间)的关系为or, 列表元素之间的关系为and
    limits：[[[0, 10, 1, 1], [15, 30, 1, 1]], [5, 20, 0, 0]]
    tips:
    如果同时给出another_limits(不推荐),则会求limits和another_limits的交集
    '''
    
    if another_limits!=None:
        limits=[limits, another_limits]
    #bug: np.array([[[1,2,3], [4,5,6]], [1,5,6]]).ndim=1   wjx:2021-8-10
    if np.array(limits).ndim==1 and (type(limits[0]) in [int, float]):
        limits=[limits]
    limit=[]
    for limit_i in limits:
        if type(limit_i)==list:
            limit=UnionSet2(limit, limit_i)
#         else:   #如果limits中包含非list值处理方式  忽略?
#             raise Exception('The given parameter should be a 2D list!')
    return limit
def UnionSet(limit1, limit2):
    '''
    description: 
    两个区间求交集, 不限制单区间(1/2维列表皆可)
    单个区间直接给出lim0=[0, 10, 1, 1],或者[[0, 10, 1, 1]](不推荐)
    多个区间必须以2D列表给出, [[0, 10, 1, 1], [20, 30, 1, 1]]
    limit1：[0, 10, 1, 1]/[[0, 10, 1, 1], [15, 30, 1, 1]]
    limit2：[5, 20, 0, 0]/...
    '''
    lim1, lim2=limit1.copy(), limit2.copy()
    dim1, dim2=np.array(limit1).ndim, np.array(limit2).ndim
    if dim1==2:
        mid=[]
        for i, lim1_i in enumerate(limit1):
            mid.append(Complement(lim1_i))
        lim1=Complement(Intersections(mid))
    if dim2==2:
        mid=[]
        for i, lim2_i in enumerate(limit2):
            mid.append(Complement(lim2_i))
        lim2=Complement(Intersections(mid))      
    return Complement(Intersection(Complement(lim1), Complement(lim2)))

94900

In [336]:
# v1

In [67]:
#字符串逻辑表达式取补集
def GetComplement1(string):
#     1：x>=10 (2 kinds)   1.1 1.2 
#     2：x<=10 (2 kinds)   2.1 2.2
#     3：10<=x<=20 (4 kinds)    3.1 3.2 3.3 3.4
#     4：x<=10 or x>=20 (4 kinds)   4.1 4.2 4.3 4.4
    count1=string.count('>')
    count2=string.count('<')
    count3=string.count('=')
    count4=string.count('or')    
    if (count1+count2==1):    #1/2
        if string[0]!='x':    #单侧约束时,不允许写10<x,必须写x>10,否则出发异常
            raise Exception('Please give the right expressions!')
        if count1==1:         #1
            if count3==0:    #1.1  x>10
                x='x<='+str(string.split('>')[-1])
            else:    
                x='x<'+str(string.split('>=')[-1])
        else: #count2==1          #2
            if count3==0:    #2.1  x<10
                x='x>='+str(string.split('<')[-1])
            else:
                x='x>'+str(string.split('<=')[-1])
    else:    #(count1+count2==2)   暂假设string里面最多只有1个or
        if count2==2: #3
            mid=string.split('x')
            if '=' in mid[0]:
                x1='x<'+mid[0].split('<=')[0]
            else:
                x1='x<='+mid[0].split('<')[0]
            if '=' in mid[1]:
                x2='x>'+mid[1].split('<=')[1]
            else:
                x2='x>='+mid[1].split('<')[1]
            x=x1+' or '+x2
        else:  #or in string
            mid=string.split(' or ')
            if '=' in mid[0]:
                x1=mid[0].split('<=')[1]+'<'
            else:
                x1=mid[0].split('<')[1]+'<='
            if '=' in mid[1]:
                x2='<'+mid[1].split('>=')[1]
            else:
                x2='<='+mid[1].split('>')[1]
            x=x1+'x'+x2
    return x

In [68]:
#字符串逻辑表达式取约束
def GetConstraints1(string):
    lim=[-float('inf'), float('inf'), 0, 0]
    lim1=[-float('inf'), float('inf'), 0, 0]
    lim2=[-float('inf'), float('inf'), 0, 0]
    count1=string.count('>')
    count2=string.count('<')
    count3=string.count('=')
    count4=string.count('or')
    if (count1+count2==1):    #仅包含单侧约束
        if string[0]!='x':    #单侧约束时,不允许写10<x,必须写x>10,否则出发异常
            raise Exception('Error1: Please give the right expressions!')
        if count1==1:         #仅包含下限
            if count3==0:     #开区间
                lim[0]=float(string.split('>')[-1])
                lim[2]=0
            else:    
                lim[0]=float(string.split('>=')[-1])
                lim[2]=1
        else: #count2==1          #仅包含上限
            if count3==0:         #开区间
                lim[1]=float(string.split('<')[-1])
                lim[3]=0
            else:
                lim[1]=float(string.split('<=')[-1])
                lim[3]=1
    else:    #(count1+count2==2)   暂假设string里面最多只有1个or
        if count2==2:     #包含两侧区间，1段
            mid=string.split('x')
            if '=' in mid[0]:   #下限闭区间  
                lim[0]=float(mid[0].split('<=')[0])
                lim[2]=1
            else:               #下限开区间
                lim[0]=float(mid[0].split('<')[0])
                lim[2]=0
            if '=' in mid[1]:   #上限闭区间  
                lim[1]=float(mid[1].split('<=')[1])
                lim[3]=1
            else:
                lim[1]=float(mid[1].split('<')[1])
                lim[3]=0
        else:  #or in string, 包含两侧区间，2段     两段区间写法必须为x<20 or x>10 不允许x>10 or x<20 否则触发异常
            mid=string.split(' or ')
            if '>' in mid[0]:
                raise Exception('Error2: Please give the right expressions!')
            if '=' in mid[0]:   #1段上限闭区间
                lim1[1]=float(mid[0].split('<=')[1])
                lim1[3]=1
            else:
                lim1[1]=float(mid[0].split('<')[1])
                lim1[3]=0
            
            if '=' in mid[1]:   #2段下限闭区间
                lim2[0]=float(mid[1].split('>=')[1])
                lim2[2]=1
            else:
                lim2[0]=float(mid[1].split('>')[1])
                lim2[2]=0
            lim=[lim1, lim2]
    return lim

In [69]:
def GetIntersection1(limit1, limit2):
    mid=[]
    if (limit1!=[]) and (limit2!=[]):
        len1, len2 = len(limit1), len(limit2)
        if len1+len2==8:
            lim1, lim2=limit1, limit2
            if (lim1[0]>lim2[1]) or (lim2[0]>lim1[1]):
                mid1=[]
            else:
                lim=[-float('inf'), float('inf'), 0, 0]
                mins=max([lim1[0], lim2[0]])
                maxs=min([lim1[1], lim2[1]])
                lim[0]=mins
                lim[1]=maxs
                if mins==lim2[0]:
                    lim[2]=lim2[2]
                else:
                    lim[2]=lim1[2]
                if maxs==lim2[1]:
                    lim[3]=lim2[3]
                else:
                    lim[3]=lim1[3]
                mid1=lim   
                if mid1!=[]: 
                    mid.append(mid1) 
        elif len1+len2==6:
            mid1=[-float('inf'), float('inf'), 0, 0]
            mid2=[-float('inf'), float('inf'), 0, 0]
            if len1==4:
                lim1, lim2=limit1, limit2
                min1, max1=lim1[0], lim1[1]
                min21, max21, min22, max22=lim2[0][0], lim2[0][1], lim2[1][0], lim2[1][1]
                if (max21>min1) or ((max21==min1) and (lim2[0][3]+lim1[2])==2):  #1段存在
                    mid1[0]=max([min21, min1])  #确定1段下限
                    if min1>min21:
                        mid1[2]=lim1[2]
                    elif min21>min1:
                        mid1[2]=lim2[0][2]
                    else:
                        mid1[2]=min([lim1[2], lim2[0][2]])

                    mid1[1]=min([max21, max1])    #确定1段上限
                    if max1>max21:
                        mid1[3]=lim2[0][3]
                    elif max21>max1:
                        mid1[3]=lim1[3]
                    else:
                        mid1[3]=min([lim2[0][3], lim1[3]])
                    if mid1!=[]: 
                        mid.append(mid1)     

                if (max1>min22) or ((max1==min22) and (lim1[3]+lim2[1][2])==2): #2段存在   
                    mid2[0]=max([min1, min22])   #确定2段下限
                    if min1>min22:
                        mid2[2]=lim1[2]
                    elif min22>min1:
                        mid2[2]=lim2[1][2]
                    else:
                        mid2[2]=min([lim1[2], lim2[1][2]])

                    mid2[1]=min([max1, max22])   #确定2段上限
                    if max1>max22:
                        mid2[3]=lim2[1][3]
                    elif max22>max1:
                        mid2[3]=lim1[3]
                    else:
                        mid2[3]=min([lim2[1][3], lim1[3]])
                    if mid1!=[]: 
                        mid.append(mid2)
            else: #len1==2 len2=4
                lim1, lim2=limit2, limit1
                min1, max1=lim1[0], lim1[1]
                min21, max21, min22, max22=lim2[0][0], lim2[0][1], lim2[1][0], lim2[1][1]
                if (max21>min1) or ((max21==min1) and (lim2[0][3]+lim1[2])==2):  #1段存在
                    mid1[0]=max([min21, min1])  #确定1段下限
                    if min1>min21:
                        mid1[2]=lim1[2]
                    elif min21>min1:
                        mid1[2]=lim2[0][2]
                    else:
                        mid1[2]=min([lim1[2], lim2[0][2]])

                    mid1[1]=min([max21, max1])    #确定1段上限
                    if max1>max21:
                        mid1[3]=lim2[0][3]
                    elif max21>max1:
                        mid1[3]=lim1[3]
                    else:
                        mid1[3]=min([lim2[0][3], lim1[3]])
                    if mid1!=[]: 
                        mid.append(mid1)     

                if (max1>min22) or ((max1==min22) and (lim1[3]+lim2[1][2])==2): #2段存在   
                    mid2[0]=max([min1, min22])   #确定2段下限
                    if min1>min22:
                        mid2[2]=lim1[2]
                    elif min22>min1:
                        mid2[2]=lim2[1][2]
                    else:
                        mid2[2]=min([lim1[2], lim2[1][2]])

                    mid2[1]=min([max1, max22])   #确定2段上限
                    if max1>max22:
                        mid2[3]=lim2[1][3]
                    elif max22>max1:
                        mid2[3]=lim1[3]
                    else:
                        mid2[3]=min([lim2[1][3], lim1[3]])
                    if mid2!=[]: 
                        mid.append(mid2)          
        else: #len1+len2==4
            if (limit1[0][1]>limit1[1][0]) or (limit2[0][1]>limit2[1][0]): # tips: 两段区间小区间写前面  大区间写后面  不允许写成[20, 30], [0, 10] 否则触发异常
                raise Exception('Please give the right expressions!')     
            lim1, lim2=limit1[0], limit2[0]
            min1, max1, min2, max2=lim1[0], lim1[1], lim2[0], lim2[1]
            mid1=[-float('inf'), float('inf'), 0, 0]
            mid2=[-float('inf'), float('inf'), 0, 0]

            if (lim1[0]>lim2[1]) or (lim2[0]>lim1[1]):
                mid1=[]
            else:
                lim=[-float('inf'), float('inf'), 0, 0]
                mins=max([lim1[0], lim2[0]])
                maxs=min([lim1[1], lim2[1]])
                lim[0]=mins
                lim[1]=maxs
                if mins==lim2[0]:
                    lim[2]=lim2[2]
                else:
                    lim[2]=lim1[2]
                if maxs==lim2[1]:
                    lim[3]=lim2[3]
                else:
                    lim[3]=lim1[3]
                mid1=lim
            if mid1!=[]: 
                mid.append(mid1)
            
            lim1, lim2=limit1[0], limit2[1]
            min1, max1, min2, max2=lim1[0], lim1[1], lim2[0], lim2[1]
            mid1=[-float('inf'), float('inf'), 0, 0]
            mid2=[-float('inf'), float('inf'), 0, 0]

            if (lim1[0]>lim2[1]) or (lim2[0]>lim1[1]):
                mid1=[]
            else:
                lim=[-float('inf'), float('inf'), 0, 0]
                mins=max([lim1[0], lim2[0]])
                maxs=min([lim1[1], lim2[1]])
                lim[0]=mins
                lim[1]=maxs
                if mins==lim2[0]:
                    lim[2]=lim2[2]
                else:
                    lim[2]=lim1[2]
                if maxs==lim2[1]:
                    lim[3]=lim2[3]
                else:
                    lim[3]=lim1[3]
                mid1=lim
            if mid1!=[]: 
                mid.append(mid1)
            
            lim1, lim2=limit1[1], limit2[0]
            min1, max1, min2, max2=lim1[0], lim1[1], lim2[0], lim2[1]
            mid1=[-float('inf'), float('inf'), 0, 0]
            mid2=[-float('inf'), float('inf'), 0, 0]

            if (lim1[0]>lim2[1]) or (lim2[0]>lim1[1]):
                mid1=[]
            else:
                lim=[-float('inf'), float('inf'), 0, 0]
                mins=max([lim1[0], lim2[0]])
                maxs=min([lim1[1], lim2[1]])
                lim[0]=mins
                lim[1]=maxs
                if mins==lim2[0]:
                    lim[2]=lim2[2]
                else:
                    lim[2]=lim1[2]
                if maxs==lim2[1]:
                    lim[3]=lim2[3]
                else:
                    lim[3]=lim1[3]
                mid1=lim
            if mid1!=[]: 
                mid.append(mid1)
            
            lim1, lim2=limit1[1], limit2[1]
            min1, max1, min2, max2=lim1[0], lim1[1], lim2[0], lim2[1]
            mid1=[-float('inf'), float('inf'), 0, 0]
            mid2=[-float('inf'), float('inf'), 0, 0]

            if (lim1[0]>lim2[1]) or (lim2[0]>lim1[1]):
                mid1=[]
            else:
                lim=[-float('inf'), float('inf'), 0, 0]
                mins=max([lim1[0], lim2[0]])
                maxs=min([lim1[1], lim2[1]])
                lim[0]=mins
                lim[1]=maxs
                if mins==lim2[0]:
                    lim[2]=lim2[2]
                else:
                    lim[2]=lim1[2]
                if maxs==lim2[1]:
                    lim[3]=lim2[3]
                else:
                    lim[3]=lim1[3]
                mid1=lim
            if mid1!=[]: 
                mid.append(mid1)
            
        if len(mid)==1:
            mid=mid[0]
    return mid
def GetIntersections(limits):
    #limits必须为2维列表且包含不少于2个元素
    limit=[-float('inf'), float('inf'), 1, 1]
    for limit_i in limits:
        if type(limit_i)==list:
            limit=GetIntersection(limit, limit_i)
    if limit==[-float('inf'), float('inf'), 1, 1]:
        limit=[]
    return limit

In [4]:
82.9*2*0.88*1.2
68.9*3*0.8

175.08480000000003

165.36